Kaggle TPU [usage guide](https://www.kaggle.com/docs/tpu)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from PIL import Image, ImageDraw
import json

In [ ]:
for i in glob.glob('/kaggle/input/hubmap-kidney-segmentation/train/*.tiff'):
    print(i)

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
path = '/kaggle/input/hubmap-kidney-segmentation/train/'
filename = '095bf7a1f'
image = rasterio.open(path + filename + '.tiff', transform = identity)
with open(path + filename + '.json') as file:
    label = json.load(file) 

In [ ]:
def get_image(img, offset):
    
    i = offset[1] * IMG_SIZE
    j = offset[0] * IMG_SIZE
    image = img.read([1,2,3], window=((i, i + IMG_SIZE),(j, j + IMG_SIZE)))
    
    return np.moveaxis(image, 0, -1)

def transform(x):
    
    x[0] -= offset[0] * IMG_SIZE
    x[1] -= offset[1] * IMG_SIZE
    
    return tuple(x)

def get_label(polygon):
    
    geom = polygon['geometry']['coordinates'][0]
    tup_arr = list(map(lambda x: transform(x), geom))
    
    return tup_arr

def get_overlay(image, label):
    
    img_copy = image.copy()
    draw = ImageDraw.Draw(img_copy)
    draw.polygon(label, fill = "wheat")
    img_overlay = Image.blend(image, img_copy, 0.5)
    
    return img_overlay

In [ ]:
POLY_NUM = 15
IMG_SIZE = 1024

In [ ]:
geometry = label[POLY_NUM]['geometry']['coordinates'][0]
offset = np.array(geometry).min(axis=0) // IMG_SIZE
image_slice = get_image(image, offset)
arr = get_label(label[POLY_NUM])
pil_img = Image.fromarray(image_slice)
img_with_label = get_overlay(pil_img, arr)

In [ ]:
plt.figure(figsize=(20, 20))

plt.subplot(121)
plt.imshow(pil_img)
plt.title('Original Image')

plt.subplot(122)
plt.imshow(img_with_label)
plt.title('Image with segmentation');